In [4]:
import ndcctools.taskvine as vine
import json
import os

In [5]:
!ls data/parquet/10split

group_1.parquet  group_3.parquet  group_6.parquet  group_9.parquet
group_10.parquet group_4.parquet  group_7.parquet
group_2.parquet  group_5.parquet  group_8.parquet


In [33]:
import pandas as pd
df = pd.DataFrame()
directory = 'data/parquet/10split'
for parquet in os.listdir(directory):
    if parquet.endswith('.parquet'):
        tempDf = pd.read_parquet(os.path.join(directory, parquet))
        df = pd.concat([df, tempDf])

df = df.reset_index(drop=True)
df   

,photo_url,latitude,longitude,taxon_id,quality_grade,observed_on,ancestry,name,group
0,http://inaturalist-open-data.s3.amazonaws.com/...,-13.055763,-71.546152,1351802.0,research,2022-10-01,48460/1/47120/372739/47158/184884/47157/47224/...,Abananote erinome11,8
1,http://inaturalist-open-data.s3.amazonaws.com/...,-10.707797,-75.246225,1351802.0,research,2021-09-17,48460/1/47120/372739/47158/184884/47157/47224/...,Abananote erinome11,8
2,http://inaturalist-open-data.s3.amazonaws.com/...,-10.707797,-75.246225,1351802.0,research,2021-09-17,48460/1/47120/372739/47158/184884/47157/47224/...,Abananote erinome11,8
3,http://inaturalist-open-data.s3.amazonaws.com/...,-13.055763,-71.546152,1351802.0,research,2022-10-01,48460/1/47120/372739/47158/184884/47157/47224/...,Abananote erinome11,8
4,http://inaturalist-open-data.s3.amazonaws.com/...,-10.707797,-75.246225,1351802.0,research,2021-09-17,48460/1/47120/372739/47158/184884/47157/47224/...,Abananote erinome11,8
...,...,...,...,...,...,...,...,...,...
99995,http://inaturalist-open-data.s3.amazonaws.com/...,39.313147,-82.104272,56061.0,research,2021-04-24,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata27,4
99996,http://inaturalist-open-data.s3.amazonaws.com/...,43.408011,-80.504497,56061.0,research,2018-05-18,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata27,4
99997,http://inaturalist-open-data.s3.amazonaws.com/...,45.034758,-76.810933,56061.0,research,2022-04-15,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata27,4
99998,http://inaturalist-open-data.s3.amazonaws.com/...,40.148471,-83.031501,56061.0,research,2023-03-18,48460/47126/211194/47125/47124/71275/47204/147...,Alliaria petiolata27,4


0      http://inaturalist-open-data.s3.amazonaws.com/...
1      http://inaturalist-open-data.s3.amazonaws.com/...
2      http://inaturalist-open-data.s3.amazonaws.com/...
3      http://inaturalist-open-data.s3.amazonaws.com/...
4      http://inaturalist-open-data.s3.amazonaws.com/...
                             ...                        
995    http://inaturalist-open-data.s3.amazonaws.com/...
996    http://inaturalist-open-data.s3.amazonaws.com/...
997    http://inaturalist-open-data.s3.amazonaws.com/...
998    http://inaturalist-open-data.s3.amazonaws.com/...
999    http://inaturalist-open-data.s3.amazonaws.com/...
Name: photo_url, Length: 1000, dtype: object

In [37]:
import asyncio
import aiohttp
from urllib.parse import urlsplit, urlunsplit

async def estimate_image_size(session, url):
    try:
        # Send a HEAD request to get headers only
        async with session.head(url) as response:
            # Check if the request was successful and contains Content-Length
            if response.status == 200 and 'Content-Length' in response.headers:
                size_in_bytes = int(response.headers['Content-Length'])
                size_in_kb = size_in_bytes / 1024
                return size_in_kb
            else:
                return None
    except Exception:
        return None

async def try_different_extensions(session, url):
    # Try the original URL first
    size_kb = await estimate_image_size(session, url)
    if size_kb is not None:
        return size_kb

    # If the original URL doesn't work, try different extensions
    extensions = ['.JPG', '.jpeg', '.JPEG', '.jpg']
    url_parts = urlsplit(url)
    
    for ext in extensions:
        # Replace the extension in the URL
        base, current_ext = url_parts.path.rsplit('.', 1)
        new_url = urlunsplit((url_parts.scheme, url_parts.netloc, f"{base}{ext}", url_parts.query, url_parts.fragment))
        print(f"Retrying with {new_url}...")
        
        size_kb = await estimate_image_size(session, new_url)
        if size_kb is not None:
            return size_kb

    # Return 0 if none of the extensions worked
    print(f"Could not determine size for {url}")
    return 0

async def get_total_size(urls):
    total_size_kb = 0
    async with aiohttp.ClientSession() as session:
        tasks = [try_different_extensions(session, url) for url in urls]
        sizes_kb = await asyncio.gather(*tasks)

    total_size_kb = sum(sizes_kb)
    total_size_mb = total_size_kb / 1024
    total_size_gb = total_size_mb / 1024
    return total_size_kb, total_size_mb, total_size_gb

# Running the asyncio loop
async def run(urls):
    total_size_kb, total_size_mb, total_size_gb = await get_total_size(urls)
    print(f"Total Size: {total_size_mb:.2f} MB ({total_size_gb:.2f} GB) ")

# Now call run with await
await run(df['photo_url'])

Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3048616/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3176245/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/2934899/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/2942227/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/2934898/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/2777271/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3176243/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/2942226/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3052925/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/3053014/large.JPG...
Retrying with http://inaturalist-open-data.s3.amazonaws.com/photos/2771433/large.JPG...
Retrying with http://inaturalist

In [3]:
import subprocess

def get_local_ip():
    try:
        # Run the 'hostname -I' command
        result = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # Get the output, strip any extra spaces or newlines
        ip_addresses = result.stdout.strip()
        return ip_addresses
    except subprocess.CalledProcessError as e:
        return f"Error: {e}"

# Get and print the local IP addresses
local_ips = get_local_ip()
print(f"My local IP addresses are: {local_ips}")
print(local_ips.split()[0])
managerIp = local_ips.split()[0]

My local IP addresses are: 10.140.86.137 169.254.95.120 10.0.86.137
10.140.86.137


In [4]:
slurm_script = """#!/bin/bash
#SBATCH --job-name=taskvine           
#SBATCH --output=result.out           
#SBATCH --account=nirav
#SBATCH --partition=standard           
#SBATCH --nodes=1
#SBATCH --ntasks=2                    
#SBATCH --time=01:00:00               

# Load any necessary modules
module load python/3.9
source ~/.bashrc
conda init bash
conda init
conda activate taskvine

# Execute the Python script or command
vine_submit_workers -T slurm -p "--cpus-per-task=4 --time=1:00:00" -t 01:00:00 """+managerIp+""" 9124 10

"""

In [5]:
slurm_filename = "taskvine.slurm"
with open(slurm_filename, "w") as slurm_file:
    slurm_file.write(slurm_script)

print(f"SLURM script saved as {slurm_filename}")

SLURM script saved as taskvine.slurm


In [6]:
%%bash
python bin/TaskvineLDAWT.py &
sbatch taskvine.slurm

Submitted batch job 2146022
Listening on port 9124
Waiting for tasks to complete...
100%|██████████| 10000/10000 [01:04<00:00, 154.96it/s]
Completed with 78 errors.
Tared output folder into: /tmp/worker-2299-11626/task.8/group_8.tar.gz

100%|██████████| 10000/10000 [00:47<00:00, 211.49it/s]
Completed with 55 errors.
Tared output folder into: /tmp/worker-2299-19734/task.9/group_6.tar.gz

100%|██████████| 10000/10000 [01:43<00:00, 97.00it/s]
Completed with 83 errors.
Tared output folder into: /tmp/worker-2299-9400/task.10/group_9.tar.gz

100%|██████████| 10000/10000 [00:42<00:00, 235.81it/s]
Completed with 58 errors.
Tared output folder into: /tmp/worker-2299-1339/task.3/group_4.tar.gz

100%|██████████| 10000/10000 [01:10<00:00, 140.94it/s]
Completed with 49 errors.
Tared output folder into: /tmp/worker-2299-11624/task.4/group_7.tar.gz

100%|██████████| 10000/10000 [01:57<00:00, 84.96it/s]
Completed with 44 errors.
Tared output folder into: /tmp/worker-2299-9401/task.2/group_5.tar.gz

10